# **Nutritional Profiling and Clustering of Food Products Using OpenFoodFacts**

## **1. Introduction**

This project explores **nutritional data analysis** using the [OpenFoodFacts](https://world.openfoodfacts.org/) public API.  
By extracting, cleaning, and storing product information in a **SQLite database**, we create a structured dataset of food items and their nutritional values.  

Key objectives include:  
- **Building a database schema** to organize product and nutrient information.  
- **Exploring and querying** the data to uncover insights (e.g., top protein-rich foods, sugar outliers).  
- **Classifying products** based on nutritional thresholds to evaluate their healthiness.  
- **Applying clustering algorithms (DBSCAN)** to discover natural groupings among products.  

## **2. Imports**

In [2]:
import requests           # For API calls to OpenFoodFacts
import pandas as pd       # For data handling and analysis
import sqlite3            # For interacting with the SQLite database

from sklearn.preprocessing import StandardScaler  # For feature scaling
from sklearn.cluster import DBSCAN                # For clustering analysis

## **3. Database Schema**

Before diving into analysis, we’ll build a **structured database*.  
- **products**: jars on the shelf (basic details: name, brand, category, country).  
- **nutrients**: the detailed labels (numeric values per 100 g).  
- **nutrient_levels**: quick traffic-light indicators (low/medium/high + grade).  


In [306]:
# Connect to (or create) the database file
conn = sqlite3.connect("openfoodfacts.db")
cur = conn.cursor()

cur.executescript("""
-- Table: products
-- Stores basic product details retrieved from OpenFoodFacts.
CREATE TABLE IF NOT EXISTS products (
    id INTEGER PRIMARY KEY,    -- Internal unique ID for our database
    code TEXT UNIQUE,          -- Product code from OpenFoodFacts (unique identifier)
    product_name TEXT,         -- Name of the product (e.g., 'Cut Green Beans')
    brands TEXT,               -- Brand(s) of the product
    categories TEXT,           -- Categories or food groups assigned to the product
    countries TEXT,            -- Countries where the product is sold
    image_url TEXT             -- URL of the product image
);

-- Table: nutrients
-- Stores detailed nutritional values per 100 g of product.
CREATE TABLE IF NOT EXISTS nutrients (
    product_id INTEGER PRIMARY KEY,  -- Links to products.id
    energy_kj_100g REAL,             -- Energy in kilojoules per 100 g
    energy_kcal_100g REAL,           -- Energy in kilocalories per 100 g
    fat_100g REAL,                   -- Total fat per 100 g
    saturated_fat_100g REAL,         -- Saturated fat per 100 g
    carbohydrates_100g REAL,         -- Carbohydrates per 100 g
    sugars_100g REAL,                -- Sugars per 100 g
    fiber_100g REAL,                 -- Fiber per 100 g
    proteins_100g REAL,              -- Proteins per 100 g
    salt_100g REAL,                  -- Salt per 100 g
    sodium_100g REAL,                -- Sodium per 100 g
    nutrition_score_fr_100g REAL,    -- Nutrition score (France system)
    FOREIGN KEY (product_id) REFERENCES products (id)
);

-- Table: nutrient_levels
-- Stores qualitative ratings for key nutrients and nutrition grade letters.
CREATE TABLE IF NOT EXISTS nutrient_levels (
    product_id INTEGER PRIMARY KEY,  -- Links to products.id
    fat TEXT,                        -- Level of fat (e.g., 'low', 'moderate', 'high')
    saturated_fat TEXT,              -- Level of saturated fat
    sugars TEXT,                     -- Level of sugars
    salt TEXT,                       -- Level of salt
    nutrition_grades TEXT,           -- Nutrition grade (e.g., A–E)
    FOREIGN KEY (product_id) REFERENCES products (id)
);
""")

# Commit changes and close the connection
conn.commit()
conn.close()

## **4. Data Extraction**
Now we’ll **extract product data** from the OpenFoodFacts API.  
- Each request pulls canned food items sold in the United States.  
- All pages are combined into `all_products` for later cleaning and storage.

In [ ]:
# -----------------------------
# Step 1 : Imports y parámetros
# -----------------------------

# --- API configuration ---
URL = "https://world.openfoodfacts.org/api/v2/search"
CATEGORY = "en:canned-vegetables"                      # Category tag
COUNTRY = "en:united-states"                           # Country tag
USER_AGENT = "MiPortfolio/1.0 (email@example.com)"  # OFF to identify 

# --- Pagination settings ---
page = 1
page_size = 100      # Max per page (can go up to 100)
all_products = []    # List to store all retrieved products

# --- Loop through API pages ---
while True:
    params = {
        "categories_tags": CATEGORY,  # Filter by category
        "countries_tags": COUNTRY,    # Filter by country
        "page": page,                 # Current page number
        "page_size": page_size        # Number of products per page
    }
    
    # Send GET request to the API
    r = requests.get(URL, params=params, headers={"User-Agent": USER_AGENT})
    r.raise_for_status()              # Raise error if the response is not OK
    
    data = r.json()                   # Parse JSON response
    products = data.get("products", [])
    
    # Stop if no more products are returned
    if not products:
        break
    
    # Add products from this page to our list
    all_products.extend(products)
    
    # Go to next page
    page += 1
    
    # If we have iterated past the total count, exit
    if page > (data.get("count", 0) // page_size) + 1:
        break

# --- Check total products retrieved ---
print(f"Total products collected: {len(all_products)}")

## **5. Data Cleaning**

We **clean and organize the raw API data** so it’s ready to be inserted into our database.  

- We convert the messy JSON payload into **tidy DataFrames**.  
- Empty strings are replaced with `None` for consistency.  
- Product details, numeric nutrients, and qualitative nutrient levels are **flattened** into separate tables.  
- Column names are standardized (hyphens → underscores) to match our database schema.  

In [242]:
# -----------------------------
# Step 1: Extract raw products
# -----------------------------
df_products_raw = pd.DataFrame(all_products)
df_products_raw.replace({"": None}, inplace=True)

In [244]:
# -----------------------------
# Step 2: Products summary
# -----------------------------
summary_cols = ['code', 'product_name', 'brands', 'categories', 'countries', 'image_url']
df_products_summary = df_products_raw[summary_cols].copy()
df_products_summary = df_products_summary.replace('', None)

In [246]:
# -----------------------------
# Step 3: Flatten nutriments
# -----------------------------
nutriment_cols = [
    "energy-kcal_100g",
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
    "sodium_100g",
    "nutrition-score-fr_100g",
]


nutriments_flat = pd.json_normalize(df_products_raw['nutriments']).reindex(columns=nutriment_cols)
nutriments_flat = nutriments_flat.apply(pd.to_numeric, errors='coerce')
df_nutrients = pd.concat([df_products_raw[['code']], nutriments_flat], axis=1)
df_nutrients.columns = df_nutrients.columns.str.replace('-', '_')

In [248]:
# -----------------------------
# Step 4: Flatten nutrient levels + nutrition grade
# -----------------------------
levels_flat = pd.json_normalize(df_products_raw['nutrient_levels'])

df_nutri_score = pd.concat([
    df_products_raw[['code']],
    levels_flat.astype('category'),
    df_products_raw[['nutrition_grades']].astype('category')
], axis=1)
df_nutri_score.columns = df_nutri_score.columns.str.replace('-', '_')

## 6. **Storage in SQLite**

In [250]:
# Connect to the SQLite database
conn = sqlite3.connect("openfoodfacts.db")

# --- PRODUCTS ---
# Remove duplicate products by 'code' to avoid inserting duplicates
df_products_clean = df_products_summary.drop_duplicates(subset=["code"])
df_products_clean.to_sql("products", conn, if_exists="append", index=False)
product_map = pd.read_sql("SELECT id, code FROM products", conn)

# --- NUTRIENTS ---
# Merge nutrient data with the product ID map, drop 'code', and rename ID column
df_nutrients_merged = (
    df_nutrients
    .merge(product_map, on="code")
    .drop(columns=["code"])
    .rename(columns={"id": "product_id"})
)
# Append nutrient data to the 'nutrients' table
df_nutrients_merged.to_sql("nutrients", conn, if_exists="append", index=False)

# --- NUTRIENT LEVELS ---
# Merge nutrient-level data with the product ID map, drop 'code', and rename ID column
df_nutri_score_merged = (
    df_nutri_score
    .merge(product_map, on="code")
    .drop(columns=["code"])
    .rename(columns={"id": "product_id"})
)
# Append nutrient-level data to the 'nutrient_levels' table
df_nutri_score_merged.to_sql("nutrient_levels", conn, if_exists="append", index=False)

# Close the database connection to save changes
conn.close()

## **7. Data Exploration and Analysis**

In this section, we perform **SQL queries** on our database to explore product information, nutritional values, and meaningful insights.  
These queries include examples such as:  
- Retrieving product details and summaries.  
- Filtering by specific brands or categories.  
- Joining product and nutrient tables for detailed nutrition profiles.  
- Ranking items by calories, protein, or sugar content.  
- Classifying products as *Healthy*, *Neutral*, or *Unhealthy* using conditional logic.  
- Creating views and aggregations (e.g., “calorie king” per country).  

In [312]:
# --- Connect to the SQLite database ---
conn = sqlite3.connect("openfoodfacts.db")
cur = conn.cursor()

# --- SQL query: get the first 10 products with their brands and countries ---
query = """
SELECT product_name, brands, countries
FROM products
LIMIT 10;
"""

# --- Read the query result into a DataFrame for clean display ---
df = pd.read_sql_query(query, conn)

print("First 10 Products with Brands and Countries")
print(df)


First 10 Products with Brands and Countries
                              product_name                       brands  \
0                          Cut Green Beans                    Del Monte   
1  Diced Tomatoes & Green Chilies Original  Gallo family,Conagra,RO*TEL   
2                               Sweet Corn                    Del Monte   
3        Artichoke Hearts Marinated In Oil           Kirkland Signature   
4              San Marzano Peeled Tomatoes                        Cento   
5                   Organic Diced Tomatoes           Kirkland Signature   
6           Golden Sweet Whole Kernel Corn                  Great Value   
7                   Raw Organic Sauerkraut                    Wildbrine   
8                        100% Pure Pumpkin                      Libby's   
9                                    Diced                        Hunts   

                                         countries  
0                          United States,Hong Kong  
1                       

In [314]:
# --- Connect to the SQLite database ---
conn = sqlite3.connect("openfoodfacts.db")
cur = conn.cursor()

# --- SQL query: search for products from the brand "Del Monte" ---
query = """
SELECT product_name, brands, countries
FROM products
WHERE LOWER(brands) LIKE '%del monte%'
ORDER BY product_name ASC;
"""

# --- Execute the query and load the results into a DataFrame ---
df = pd.read_sql_query(query, conn)

print("Products from brand: Del Monte")
print(df)

# Close the database connection to save changes
conn.close()

Products from brand: Del Monte
                                         product_name  \
0                               Asparagus cuts & tips   
1   California style vegetable & bean blends, cali...   
2                                     Chopped spinach   
3   Classic style vegetable & bean blends, classic...   
4   Country style vegetable & bean blends, country...   
..                                                ...   
70  Whole kernel sweet fiesta corn seasoned with r...   
71               Whole kernel sweet gold & white corn   
72                      Whole kernel sweet white corn   
73              Whole kernel sweet white shoepeg corn   
74                                     Whole potatoes   

                                            brands      countries  
0   Del Monte Quality,  Del Monte Foods, Del Monte  United States  
1                                        Del Monte  United States  
2                    Allens,  Del Monte Foods Inc.  United States  
3           

In [316]:
# --- Connect to the SQLite database ---
conn = sqlite3.connect("openfoodfacts.db")
cur = conn.cursor()

# --- SQL query: join products and nutrients to list nutritional information ---
query = """
SELECT 
    p.product_name,
    p.brands,
    n.energy_kcal_100g AS calories,
    n.fat_100g AS fat,
    n.sugars_100g AS sugars,
    n.proteins_100g AS proteins
FROM products AS p
JOIN nutrients AS n
    ON p.id = n.product_id
ORDER BY calories DESC
LIMIT 20;
"""

# --- Execute the query and load the results into a DataFrame ---
df = pd.read_sql_query(query, conn)

print("Top 20 Products with Nutritional Information (Highest Calories First)")
print(df)

# Close the database connection to save changes
conn.close()

Top 20 Products with Nutritional Information (Highest Calories First)
                                         product_name  \
0                    Mixed antipesto in sunflower oil   
1                     Whole foods market, green beans   
2               Whole foods market, green beans chips   
3                         Grilled artichokes, grilled   
4                                       Mushrooms Mix   
5                                    Green split peas   
6                    Triple cleaned yellow split peas   
7                         Southern bbq boiled peanuts   
8             Sempio, braised lotus root in soy sauce   
9                    Poivrons Grillés Pour Bruschetta   
10                                   Green split peas   
11                            Pumpkin Chunks In Syrup   
12                                   Sliced Mushrooms   
13                            Mushroom Pieces & Stems   
14                               Whole white potatoes   
15                

In [318]:
# --- Connect to the SQLite database ---
conn = sqlite3.connect("openfoodfacts.db")
cur = conn.cursor()

# --- SQL query: rank the top 5 products by protein content (g/100g) ---
query = """
SELECT 
    p.product_name,
    p.brands,
    n.proteins_100g AS proteins
FROM products AS p
JOIN nutrients AS n
    ON p.id = n.product_id
WHERE n.proteins_100g IS NOT NULL
ORDER BY n.proteins_100g DESC
LIMIT 5;
"""

# --- Execute the query and load the results into a DataFrame ---
df = pd.read_sql_query(query, conn)

print("Top 5 Products with Highest Protein Content (g/100g):")
print(df)

# Close the database connection to save changes
conn.close()

Top 5 Products with Highest Protein Content (g/100g):
                       product_name brands  proteins
0                 Piselli Finissimi   None     49.00
1                  Green split peas   None     24.00
2  Triple cleaned yellow split peas   None     24.00
3                  Green split peas   None     22.86
4                 Yellow split peas   None     20.00


In [332]:
# --- Connect to the SQLite database ---
conn = sqlite3.connect("openfoodfacts.db")
cur = conn.cursor()

# --- SQL query: Count products grouped by individual country ---
query = """
WITH RECURSIVE split AS (
    SELECT
        id,
        TRIM(SUBSTR(countries, 1, INSTR(countries || ',', ',') - 1)) AS country,
        SUBSTR(countries || ',', INSTR(countries || ',', ',') + 1) AS rest
    FROM products
    WHERE countries IS NOT NULL AND countries <> ''
    UNION ALL
    SELECT
        id,
        TRIM(SUBSTR(rest, 1, INSTR(rest, ',') - 1)),
        SUBSTR(rest, INSTR(rest, ',') + 1)
    FROM split
    WHERE rest <> ''
)
SELECT
    country,
    COUNT(DISTINCT id) AS product_count
FROM split
WHERE country <> ''
GROUP BY country
ORDER BY product_count DESC;
"""

# --- Execute the query and load the results into a DataFrame ---
df = pd.read_sql_query(query, conn)

print("Products by Country of Origin:")
print(df)

# Close the database connection to save changes
conn.close()

Products by Country of Origin:
             country  product_count
0      United States           2976
1              en:us            109
2              World            105
3            Germany             61
4   en:United States             55
..               ...            ...
71            Brazil              1
72            Brasil              1
73           Belgien              1
74         Australië              1
75   Arabie saoudite              1

[76 rows x 2 columns]


In [364]:
# --- Connect to the SQLite database ---
conn = sqlite3.connect("openfoodfacts.db")
cur = conn.cursor()

# --- SQL query: classify products as Healthy, Neutral, or Unhealthy based on thresholds ---
query = """
SELECT 
  p.product_name,
  p.brands,
  CASE
    WHEN n.energy_kcal_100g <150 AND n.fat_100g <5 
         AND n.sugars_100g <5 AND n.salt_100g <0.3 THEN 'Healthy'
    WHEN n.energy_kcal_100g >400 OR n.fat_100g >20 
         OR n.sugars_100g >25 OR n.salt_100g >1.5 THEN 'Unhealthy'
    ELSE 'Neutral'
  END AS classification
FROM products p
JOIN nutrients n ON p.id = n.product_id
WHERE n.energy_kcal_100g IS NOT NULL 
  AND n.fat_100g IS NOT NULL 
  AND n.sugars_100g IS NOT NULL 
  AND n.salt_100g IS NOT NULL
ORDER BY classification, p.product_name;
"""

# --- Execute query and load into DataFrame ---
df = pd.read_sql_query(query, conn)

print("Sample of 20 Products Classified by Nutritional Profile")
print(df.sample(n=20, random_state=7))

# Close the database connection to save changes
conn.close()

Sample of 20 Products Classified by Nutritional Profile
                                           product_name        brands  \
1420                                  Green pigeon peas          None   
2708           Whole kernel supersweet corn, supersweet          None   
751                                Classic 3 bean salad          None   
1947                             Pre-cut hearts of palm        Roland   
2957                                       Pepperoncini   Giant Eagle   
2463                                Verde Diced Chilies  Brookshire's   
2194                                   Sliced mushrooms          None   
2182                                 Sliced green beans          None   
1395                                        Green beans          None   
1085                                     Diced Tomatoes     Hannaford   
1929                                 Pickled Red Onions      Mezzetta   
2556                            Whole Kernel Sweet Corn   Happy Bell

In [366]:
# --- Connect to the SQLite database ---
conn = sqlite3.connect("openfoodfacts.db")
cur = conn.cursor()

# --- SQL query: find products whose sugar content is ≥50% above the overall average ---
query = """
SELECT 
  p.product_name,
  p.brands,
  n.sugars_100g
FROM products p
JOIN nutrients n
  ON p.id = n.product_id
WHERE n.sugars_100g IS NOT NULL
  AND n.sugars_100g >= 1.5 * (
      SELECT AVG(sugars_100g) 
      FROM nutrients 
      WHERE sugars_100g IS NOT NULL
  )
ORDER BY n.sugars_100g DESC;
"""

# --- Execute query and load results into a DataFrame ---
df = pd.read_sql_query(query, conn)

print("Products with Sugar ≥50% Above the Average:")
print(df)

# Close the database connection to save changes
conn.close()

Products with Sugar ≥50% Above the Average:
                                product_name                   brands  \
0                    Pumpkin Chunks In Syrup  Dulces Reg. La Molienda   
1    Sempio, braised lotus root in soy sauce                   Sempio   
2                        Sweet whole pickles                DEL MONTE   
3                        Horseradish pickles            Amish Wedding   
4                               Candied Yams              Glory Foods   
..                                       ...                      ...   
671                          Foods baby corn                   Roland   
672                            Cut baby corn                     None   
673                     Whole white potatoes              Always Save   
674                                  Pumpkin                    Polar   
675                      All natural pumpkin                 Mw Polar   

     sugars_100g  
0      61.730000  
1      43.330000  
2      35.710000  
3  

In [368]:
# --- Connect to the SQLite database ---
conn = sqlite3.connect("openfoodfacts.db")
cur = conn.cursor()

# --- SQL query: find the highest-calorie product ("calorie king") for each country ---
query = """
WITH RECURSIVE
split AS (
  -- Split comma-separated country lists into individual rows
  SELECT id,
         TRIM(SUBSTR(countries, 1, INSTR(countries||',', ',')-1)) AS country,
         SUBSTR(countries||',', INSTR(countries||',', ',')+1) AS rest
  FROM products
  WHERE countries IS NOT NULL AND countries <> ''
  UNION ALL
  SELECT id,
         TRIM(SUBSTR(rest, 1, INSTR(rest, ',')-1)),
         SUBSTR(rest, INSTR(rest, ',')+1)
  FROM split
  WHERE rest <> ''
),
per_country AS (
  -- Rank products by energy for each country
  SELECT s.country,
         p.product_name,
         p.brands,
         n.energy_kcal_100g,
         ROW_NUMBER() OVER (PARTITION BY s.country ORDER BY n.energy_kcal_100g DESC) AS rn
  FROM split s
  JOIN products p ON p.id = s.id
  JOIN nutrients n ON n.product_id = p.id
  WHERE n.energy_kcal_100g IS NOT NULL AND s.country <> ''
)
-- Select only the top-ranked product per country
SELECT country, product_name, brands, energy_kcal_100g
FROM per_country
WHERE rn = 1
ORDER BY energy_kcal_100g DESC;
"""

# --- Execute query and load into DataFrame ---
df = pd.read_sql_query(query, conn)

print("Top-Calorie Product per Country (Calorie King)")
print(df.head(10))

# Close the database connection to save changes
conn.close()

Top-Calorie Product per Country (Calorie King)
                          country                      product_name  \
0                   United States  Mixed antipesto in sunflower oil   
1                          France  Poivrons Grillés Pour Bruschetta   
2                     Deutschland           Tafelmeerrettich scharf   
3  Vereinigte Staaten von Amerika           Tafelmeerrettich scharf   
4                      en:germany           Tafelmeerrettich scharf   
5                en:United States                  Artichoke Hearts   
6             Polynésie française               Sweet whole pickles   
7                           World       Dolmas Stuffed Grape Leaves   
8                      États-Unis               Horseradish pickles   
9                en:united-states       Bread & Butter Pickle Chips   

                  brands  energy_kcal_100g  
0                   None             500.0  
1         Italiamo, Lidl             310.0  
2              K-Classic            

In [378]:
# --- Connect to the SQLite database ---
conn = sqlite3.connect("openfoodfacts.db")
cur = conn.cursor()

# --- 1) Create the view for high-fiber, high-protein products ---
cur.executescript("""
DROP VIEW IF EXISTS high_fiber_protein_products;
CREATE VIEW high_fiber_protein_products AS
SELECT 
  p.product_name,
  p.brands,
  n.fiber_100g,
  n.proteins_100g
FROM products p
JOIN nutrients n ON p.id = n.product_id
WHERE n.fiber_100g >= 25    -- adjust threshold if needed
  AND n.proteins_100g >= 20 -- adjust threshold if needed
ORDER BY n.fiber_100g DESC, n.proteins_100g DESC;
""")
conn.commit()

# --- 2) Query the view and load into a DataFrame ---
df = pd.read_sql_query(
    "SELECT * FROM high_fiber_protein_products;",
    conn
)

print("High-Fiber, High-Protein Products")
print(df)

# --- Close the database connection ---
conn.close()

High-Fiber, High-Protein Products
                       product_name brands  fiber_100g  proteins_100g
0                 Piselli Finissimi   None        69.0           49.0
1                  Green split peas   None        26.0           24.0
2  Triple cleaned yellow split peas   None        26.0           24.0


## **8. Clustering Analysis**

In this section, we apply **unsupervised learning** to group products based on their nutritional profiles.  
Using **DBSCAN**, a density-based clustering algorithm that doesn’t require specifying the number of clusters, we:  
- Load the relevant nutrient features from our database.  
- Scale the data to ensure all variables contribute equally.  
- Detect natural groupings among products without prior labels.  
- Summarize cluster counts and show sample products from each group.  

In [381]:
# -----------------------------
# Step 1: Load data from SQLite
# -----------------------------
DB_PATH = "openfoodfacts.db"
SQL = """
SELECT
  p.id,
  p.product_name,
  p.brands,
  n.energy_kcal_100g,
  n.fat_100g,
  n.saturated_fat_100g,
  n.carbohydrates_100g,
  n.sugars_100g,
  n.proteins_100g,
  n.salt_100g
FROM products p
JOIN nutrients n ON n.product_id = p.id
WHERE n.energy_kcal_100g IS NOT NULL
  AND n.fat_100g IS NOT NULL
  AND n.saturated_fat_100g IS NOT NULL
  AND n.carbohydrates_100g IS NOT NULL
  AND n.sugars_100g IS NOT NULL
  AND n.proteins_100g IS NOT NULL
  AND n.salt_100g IS NOT NULL;
"""

conn = sqlite3.connect(DB_PATH)
df = pd.read_sql_query(SQL, conn)
conn.close()

# Features to cluster on
FEATURES = [
    "energy_kcal_100g", "fat_100g", "saturated_fat_100g",
    "carbohydrates_100g", "sugars_100g", "proteins_100g", "salt_100g"
]
X = df[FEATURES].copy()

# -----------------------------
# 2) Scale features (critical for distance-based clustering)
# -----------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# -----------------------------
# 3) DBSCAN
#    - eps: neighborhood radius in standardized units
#    - min_samples: minimum points to form a dense region
# -----------------------------
dbscan = DBSCAN(eps=1.6, min_samples=8)  # start values
labels = dbscan.fit_predict(X_scaled)
df["cluster"] = labels  # -1 denotes noise/outliers

# -----------------------------
# 4) Summary + quick peek
# -----------------------------
print("Cluster counts (−1 = noise):")
print(df["cluster"].value_counts().sort_index())

print("\nSample items per cluster:")
for c in sorted(df["cluster"].unique()):
    sample = df[df["cluster"] == c][["product_name", "brands"]].head(5)
    print(f"\nCluster {c}:")
    print(sample.to_string(index=False))


Cluster counts (−1 = noise):
cluster
-1      70
 0    2821
 1       9
 2      11
Name: count, dtype: int64

Sample items per cluster:

Cluster -1:
                               product_name             brands
                           Crushed tomatoes Whole Foods Market
                     Whole Artichoke Hearts       Trader Joe's
        Cento, eggplant appetizer, caponata              Cento
Urbani truffles, black truffles & mushrooms    Urbani Truffles
                    Whole tomatoes w/ basil          Contadina

Cluster 0:
                     product_name                                       brands
                  Cut Green Beans                                    Del Monte
Stokelys Whole Kernel Golden Corn                                    Stokely's
          Whole Kernel Sweet Corn                                  Green Giant
                       Sweet Peas                                  Great Value
                     Sliced Beets Del Monte Quality,Del Monte Foods,

- **Cluster 0 (2,821 items)** – Largest group consisting mainly of **plain canned vegetables** (e.g., green beans, corn, peas, beets). These products share very similar nutritional profiles: low fat, moderate carbs, and low salt/sugar.  
- **Cluster 1 (9 items)** – Contains **marinated artichoke products** in oil or brine. Added oils and seasonings raise fat and calorie content, making them distinct from the plain vegetable group.  
- **Cluster 2 (11 items)** – A **small group of specialty preserved vegetables** (e.g., marinated artichokes, grilled eggplant, collard greens) profiles—likely influenced by seasonings
- **Noise (-1, 70 items)** – Outliers that do not belong to any dense cluster. These could be unique recipes, incomplete data, or products with extreme nutrient values.

DBSCAN separated **basic canned vegetables** from **marinated or specialty varieties**, highlighting nutritional differences introduced by added oils and seasonings.

---